<a href="https://colab.research.google.com/github/vandislav/data_science_practicum/blob/main/genomic_benchmarks_adam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install -qq tensorflow_addons genomic-benchmarks

import tensorflow as tf
import numpy as np 

from pathlib import Path
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer, binary_f1_score
from genomic_benchmarks.models.tf import basic_cnn_model_v0 as model

from tensorflow.keras.models import Sequential


In [38]:
from genomic_benchmarks.data_check import list_datasets
 
list_datasets()

['demo_mouse_enhancers',
 'human_nontata_promoters',
 'demo_coding_vs_intergenomic_seqs',
 'human_enhancers_cohn',
 'demo_human_or_worm',
 'human_enhancers_ensembl']

In [39]:
from genomic_benchmarks.data_check import info
info('human_nontata_promoters')

/usr/local/lib/python3.7/dist-packages/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Dataset `human_nontata_promoters` has 2 classes: negative, positive.

All lenghts of genomic intervals equals 251.

Totally 36131 sequences have been found, 27097 for training and 9034 for testing.


,train,test
negative,12355,4119
positive,14742,4915


In [67]:
BATCH_SIZE = 32
SEQ_TRAIN_PATH = Path.home() / '.genomic_benchmarks' / 'human_nontata_promoters' / 'train'
CLASSES = ['negative', 'positive']
train_dset = tf.keras.preprocessing.text_dataset_from_directory(
     directory=SEQ_TRAIN_PATH,
     batch_size=BATCH_SIZE,
     class_names=CLASSES)

list(train_dset)[0][0][0]

Found 27097 files belonging to 2 classes.


<tf.Tensor: shape=(), dtype=string, numpy=b'CCGCCCCCCGCCGCGCCCCGCCCCCCGCGCCGCGCCCCGCCCCCCGCCGCGCCCCTCCCGCGCCCGCCGCCCCGCCCCTCCCGCGCCCGCCGCCGTCGTAGCGCCTAGGCGGGGCCTCTCGTAGCGACGCCCCCAGGGGGGCAGAACGCGTGCGCACGCGCGTGCGAACGCGCGTGCGCGGGCGGTGTTTGAGGCCGGCCAGTCGTGACTGGGCGGCAATGAGGTCAGTGACTGCCGGGAGTCCTGCAGGG'>

In [69]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
#vectorize_layer.set_vocabulary(vocabulary=np.asarray(['a', 'c', 't', 'g', 'n']))
vectorize_layer.get_vocabulary()

['', '[UNK]', 'g', 'c', 't', 'a']

In [70]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

In [84]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Nadam(),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0), binary_f1_score])

In [85]:
EPOCHS = 10

history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10
847/847 [==============================] - 25s 28ms/step - loss: 0.3846 - binary_accuracy: 0.8304 - f1_score: 0.8116
Epoch 2/10
847/847 [==============================] - 23s 27ms/step - loss: 0.3098 - binary_accuracy: 0.8655 - f1_score: 0.8603
Epoch 3/10
847/847 [==============================] - 24s 28ms/step - loss: 0.2854 - binary_accuracy: 0.8748 - f1_score: 0.8733
Epoch 4/10
847/847 [==============================] - 24s 28ms/step - loss: 0.2706 - binary_accuracy: 0.8826 - f1_score: 0.8807
Epoch 5/10
847/847 [==============================] - 23s 27ms/step - loss: 0.2597 - binary_accuracy: 0.8878 - f1_score: 0.8873
Epoch 6/10
847/847 [==============================] - 23s 28ms/step - loss: 0.2473 - binary_accuracy: 0.8972 - f1_score: 0.8946
Epoch 7/10
847/847 [==============================] - 22s 25ms/step - loss: 0.2433 - binary_accuracy: 0.8988 - f1_score: 0.8979
Epoch 8/10
847/847 [==============================] - 22s 26ms/step - loss: 0.2343 - binary_accuracy: 0.

In [86]:
SEQ_TEST_PATH = Path.home() / '.genomic_benchmarks' / 'human_nontata_promoters' / 'test'
CLASSES = ['negative', 'positive']
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
     directory=SEQ_TEST_PATH,
     batch_size=BATCH_SIZE,
     class_names=CLASSES)

test_ds = test_dset.map(vectorize_text)

Found 9034 files belonging to 2 classes.


In [87]:
model.evaluate(test_ds)

283/283 [==============================] - 4s 11ms/step - loss: 0.3285 - binary_accuracy: 0.8627 - f1_score: 0.8733


[0.3284582197666168, 0.8627407550811768, 0.8733079433441162]